In [1]:
BRANCH='main'

In [2]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
#!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

'\nYou can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.\n\nInstructions for setting up Colab are as follows:\n1. Open a new Python 3 notebook.\n2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)\n3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)\n4. Run this cell to set up dependencies.\n'

In [3]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-01-13 03:39:00 experimental:27] Module <function get_argmin_mat at 0x7fe31f65d3a0> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-13 03:39:00 experimental:27] Module <function getMultiScaleCosAffinityMatrix at 0x7fe31f7c8ca0> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-13 03:39:00 experimental:27] Module <function parse_scale_configs at 0x7fe31f65dd30> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022

In this tutorial, we are going to describe how to finetune BioMegatron - a [BERT](https://arxiv.org/abs/1810.04805)-like [Megatron-LM](https://arxiv.org/pdf/1909.08053.pdf) model pre-trained on large biomedical text corpus ([PubMed](https://pubmed.ncbi.nlm.nih.gov/) abstracts and full-text commercial use collection) - on the [NCBI Disease Dataset](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3951655/) for Named Entity Recognition.

The model size of Megatron-LM can be larger than BERT, up to multi-billion parameters, compared to 345 million parameters of BERT-large.
There are some alternatives of BioMegatron, most notably [BioBERT](https://arxiv.org/abs/1901.08746). Compared to BioBERT BioMegatron is larger by model size and pre-trained on larger text corpus.

A more general tutorial of using BERT-based models, including Megatron-LM, for downstream natural language processing tasks can be found [here](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/nlp/01_Pretrained_Language_Models_for_Downstream_Tasks.ipynb).

# Task Description
**Named entity recognition (NER)**, also referred to as entity chunking, identification or extraction, is the task of detecting and classifying key information (entities) in text.

For instance, **given sentences from medical abstracts, what diseases are mentioned?**<br>
In this case, our data input is sentences from the abstracts, and our labels are the precise locations of the named disease entities.  Take a look at the information provided for the dataset.

For more details and general examples on Named Entity Recognition, please refer to the [Token Classification and Named Entity Recognition tutorial notebook](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/nlp/Token_Classification_Named_Entity_Recognition.ipynb).

# Dataset

The [NCBI-disease corpus](https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/) is a set of 793 PubMed abstracts, annotated by 14 annotators. The annotations take the form of HTML-style tags inserted into the abstract text using the clearly defined rules.  The annotations identify named diseases, and can be used to fine-tune a language model to identify disease mentions in future abstracts, *whether those diseases were part of the original training set or not*.

Here's an example of what an annotated abstract from the corpus looks like:

```html
10021369	Identification of APC2, a homologue of the <category="Modifier">adenomatous polyposis coli tumour</category> suppressor .	The <category="Modifier">adenomatous polyposis coli ( APC ) tumour</category>-suppressor protein controls the Wnt signalling pathway by forming a complex with glycogen synthase kinase 3beta ( GSK-3beta ) , axin / conductin and betacatenin . Complex formation induces the rapid degradation of betacatenin . In <category="Modifier">colon carcinoma</category> cells , loss of APC leads to the accumulation of betacatenin in the nucleus , where it binds to and activates the Tcf-4 transcription factor ( reviewed in [ 1 ] [ 2 ] ) . Here , we report the identification and genomic structure of APC homologues . Mammalian APC2 , which closely resembles APC in overall domain structure , was functionally analyzed and shown to contain two SAMP domains , both of which are required for binding to conductin . Like APC , APC2 regulates the formation of active betacatenin-Tcf complexes , as demonstrated using transient transcriptional activation assays in APC - / - <category="Modifier">colon carcinoma</category> cells . Human APC2 maps to chromosome 19p13 . 3 . APC and APC2 may therefore have comparable functions in development and <category="SpecificDisease">cancer</category> .
```

In this example, we see the following tags within the abstract:
```html
<category="Modifier">adenomatous polyposis coli tumour</category>
<category="Modifier">adenomatous polyposis coli ( APC ) tumour</category>
<category="Modifier">colon carcinoma</category>
<category="Modifier">colon carcinoma</category>
<category="SpecificDisease">cancer</category>
```

For our purposes, we will consider any identified category (such as "Modifier", "Specific Disease", and a few others) to generally be a "disease".

Let's download the dataset.

In [4]:
DATA_DIR = "DATA_DIR"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, 'NER'), exist_ok=True)

In [5]:
print('Downloading NCBI data...')
wget.download('https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/NCBI_corpus.zip', DATA_DIR)
! unzip -o {DATA_DIR}/NCBI_corpus.zip -d {DATA_DIR}

Archive:  DATA_DIR/NCBI_corpus.zip
  inflating: DATA_DIR/NCBI_corpus_development.txt  
  inflating: DATA_DIR/NCBI_corpus_testing.txt  
  inflating: DATA_DIR/NCBI_corpus_training.txt  


In [6]:
# If you want to see more examples, you can explore the text of the corpus using the file browser to the left, or open files directly, for example typing a command like the following in a code-cell:

! head -1 $DATA_DIR/NCBI_corpus_testing.txt

9288106	Clustering of missense mutations in the <category="Modifier">ataxia-telangiectasia</category> gene in a <category="SpecificDisease">sporadic T-cell leukaemia</category>.	<category="SpecificDisease">Ataxia-telangiectasia</category> ( <category="SpecificDisease">A-T</category> ) is a <category="DiseaseClass">recessive multi-system disorder</category> caused by mutations in the ATM gene at 11q22-q23 ( ref . 3 ) . The risk of <category="DiseaseClass">cancer</category> , especially <category="DiseaseClass">lymphoid neoplasias</category> , is substantially elevated in <category="Modifier">A-T</category> patients and has long been associated with chromosomal instability . By analysing <category="Modifier">tumour</category> DNA from patients with <category="SpecificDisease">sporadic T-cell prolymphocytic leukaemia</category> ( <category="SpecificDisease">T-PLL</category> ) , a rare <category="DiseaseClass">clonal malignancy</category> with similarities to a <category="SpecificDisease">

We have two datasets derived from this corpus:  a text classification dataset and a named entity recognition (NER) dataset.  The text classification dataset labels the abstracts among three broad disease groupings.  We'll use this simple split to demonstrate the NLP text classification task.   The NER dataset labels individual words as diseases.  This dataset will be used for the NLP NER task.  

## Pre-process dataset
A pre-processed NCBI-disease dataset for NER can be found [here](https://github.com/spyysalo/ncbi-disease/tree/master/conll) or [here](https://github.com/dmis-lab/biobert#datasets).<br>
We download the files under {DATA_DIR/NER} directory.

In [7]:
NER_DATA_DIR = f'{DATA_DIR}/NER'
wget.download('https://raw.githubusercontent.com/spyysalo/ncbi-disease/master/conll/train.tsv', NER_DATA_DIR)
wget.download('https://raw.githubusercontent.com/spyysalo/ncbi-disease/master/conll/devel.tsv', NER_DATA_DIR)
wget.download('https://raw.githubusercontent.com/spyysalo/ncbi-disease/master/conll/test.tsv', NER_DATA_DIR)

'DATA_DIR/NER/test (1).tsv'

In [8]:
!ls -lh $NER_DATA_DIR

total 24M
-rw-r--r-- 1 root root 1.4M Jan 10 21:40  cached_text_dev.txt_BertTokenizer_128_28996_-1
-rw-r--r-- 1 root root 1.2M Jan 10 21:57  cached_text_dev.txt_BertTokenizer_128_31012_-1
-rw-r--r-- 1 root root 8.3M Jan 10 21:40  cached_text_train.txt_BertTokenizer_128_28996_-1
-rw-r--r-- 1 root root 8.2M Jan 10 21:57  cached_text_train.txt_BertTokenizer_128_31012_-1
-rw-r--r-- 1 root root 196K Jan 10 21:28  dev.tsv
-rw-r--r-- 1 root root 196K Jan 12 22:11 'devel (1).tsv'
-rw-r--r-- 1 root root 196K Jan 13 03:39  devel.tsv
-rw-r--r-- 1 root root   21 Jan 13 00:40  label_ids.csv
-rw-r--r-- 1 root root  63K Jan 12 22:12  labels_dev.txt
-rw-r--r-- 1 root root   51 Jan 10 21:40  labels_dev_label_stats.tsv
-rw-r--r-- 1 root root  65K Jan 12 22:12  labels_test.txt
-rw-r--r-- 1 root root 360K Jan 12 22:11  labels_train.txt
-rw-r--r-- 1 root root   53 Jan 13 00:40  labels_train_label_stats.tsv
-rw-r--r-- 1 root root   43 Jan 10 21:40  labels_train_weights.p
-rw-r--r-- 1 root root 201K Jan 13 0

Convert these to a format that is compatible with [NeMo Token Classification module](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/token_classification_train.py), using the [conversion script](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/data/import_from_iob_format.py).

In [9]:
! mv $NER_DATA_DIR/devel.tsv $NER_DATA_DIR/dev.tsv

In [10]:
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/nlp/token_classification/data/import_from_iob_format.py')

'import_from_iob_format (2).py'

In [11]:
! python import_from_iob_format.py --data_file=$NER_DATA_DIR/train.tsv
! python import_from_iob_format.py --data_file=$NER_DATA_DIR/dev.tsv
! python import_from_iob_format.py --data_file=$NER_DATA_DIR/test.tsv

[NeMo I 2022-01-13 03:39:12 import_from_iob_format:119] Processing DATA_DIR/NER/train.tsv
[NeMo I 2022-01-13 03:39:12 import_from_iob_format:124] Processing of the DATA_DIR/NER/train.tsv is complete
[NeMo I 2022-01-13 03:39:15 import_from_iob_format:119] Processing DATA_DIR/NER/dev.tsv
[NeMo I 2022-01-13 03:39:15 import_from_iob_format:124] Processing of the DATA_DIR/NER/dev.tsv is complete
[NeMo I 2022-01-13 03:39:18 import_from_iob_format:119] Processing DATA_DIR/NER/test.tsv
[NeMo I 2022-01-13 03:39:18 import_from_iob_format:124] Processing of the DATA_DIR/NER/test.tsv is complete


The NER task requires two files: the text sentences, and the labels.  Run the next two cells to see a sample of the two files.

In [12]:
!head $NER_DATA_DIR/text_train.txt

Identification of APC2 , a homologue of the adenomatous polyposis coli tumour suppressor . 
The adenomatous polyposis coli ( APC ) tumour - suppressor protein controls the Wnt signalling pathway by forming a complex with glycogen synthase kinase 3beta ( GSK - 3beta ) , axin / conductin and betacatenin . 
Complex formation induces the rapid degradation of betacatenin . 
In colon carcinoma cells , loss of APC leads to the accumulation of betacatenin in the nucleus , where it binds to and activates the Tcf - 4 transcription factor ( reviewed in [ 1 ] [ 2 ] ) . 
Here , we report the identification and genomic structure of APC homologues . 
Mammalian APC2 , which closely resembles APC in overall domain structure , was functionally analyzed and shown to contain two SAMP domains , both of which are required for binding to conductin . 
Like APC , APC2 regulates the formation of active betacatenin - Tcf complexes , as demonstrated using transient transcriptional activation assays in APC - / - c

In [13]:
!head $NER_DATA_DIR/labels_train.txt

O O O O O O O O B-Disease I-Disease I-Disease I-Disease O O 
O B-Disease I-Disease I-Disease I-Disease I-Disease I-Disease I-Disease O O O O O O O O O O O O O O O O O O O O O O O O O O O O O 
O O O O O O O O O 
O B-Disease I-Disease O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O 
O O O O O O O O O O O O O 
O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O 
O O O O O O O O O O O O O O O O O O O O O O O O O O B-Disease I-Disease O O 
O O O O O O O 
O O 
O O O O O O O O O O O B-Disease O 


### IOB Tagging
We can see that the abstract has been broken into sentences.  Each sentence is then further parsed into words with labels that correspond to the original HTML-style tags in the corpus. 

The sentences and labels in the NER dataset map to each other with _inside, outside, beginning (IOB)_ tagging. Anything separated by white space is a word, including punctuation.  For the first sentence we have the following mapping:

```text
Identification of APC2 , a homologue of the adenomatous polyposis coli tumour suppressor .
O              O  O    O O O         O  O   B           I         I    I      O          O  
```

Recall the original corpus tags:
```html
Identification of APC2, a homologue of the <category="Modifier">adenomatous polyposis coli tumour</category> suppressor .
```
The beginning word of the tagged text, "adenomatous", is now IOB-tagged with a <span style="font-family:verdana;font-size:110%;">B</span> (beginning) tag, the other parts of the disease, "polyposis coli tumour" tagged with <span style="font-family:verdana;font-size:110%;">I</span> (inside) tags, and everything else tagged as <span style="font-family:verdana;font-size:110%;">O</span> (outside).

# Model configuration

Our Named Entity Recognition model is comprised of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model followed by a Token Classification layer.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifier, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

## Convert the Megatron-LM Weights to Nem


In [15]:
from nemo.collections.nlp.modules.common.megatron.megatron_utils import MEGATRON_CONFIG_MAP
import pathlib
# specify BERT-like model, you want to use
PRETRAINED_BERT_MODEL = "biomegatron-bert-345m-cased"

checkpoint_url = MEGATRON_CONFIG_MAP[PRETRAINED_BERT_MODEL]['checkpoint']
vocab_url = MEGATRON_CONFIG_MAP[PRETRAINED_BERT_MODEL]['vocab']
checkpoint_filename = pathlib.Path(checkpoint_url).name
vocab_filename = pathlib.Path(vocab_url).name
if not pathlib.Path(checkpoint_filename).exists():
    print('downloading from checkpoint url', checkpoint_url)
    !wget $checkpoint_url
if not pathlib.Path(vocab_filename).exists():
    print('downloading from vocab url', vocab_url)
    !wget $vocab_url

In [17]:
WORK_DIR = "WORK_DIR"
os.makedirs(WORK_DIR, exist_ok=True)

# Prepare the model parameters 
# download the model's configuration file 
config_dir = WORK_DIR + '/configs/'
MODEL_CONFIG = "megatron_bert_config.yaml"
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/nlp/language_modeling/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

config file is already exists


In [18]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
config.model.num_layers = 24
config.model.hidden_size = 1024
config.model.ffn_hidden_size = 4096
config.model.num_attention_heads = 16
config.model.tokenizer.vocab_file = vocab_filename
config.model.tokenizer.type = 'BertWordPieceCase'
config.model.tensor_model_parallel_size = 1
config.model.data.data_prefix = ''
config.model.max_position_embeddings = 512
config.model.data.seq_length = 512
config.cfg = {}
config.cfg.cfg = config.model
with open('hparams.yaml', 'w') as f:
    f.write(OmegaConf.to_yaml(config.cfg))

WORK_DIR/configs/megatron_bert_config.yaml


In [52]:
import os
PWD = os.getcwd()
!python -m torch.distributed.run --nproc_per_node=1 ../../examples/nlp/language_modeling/megatron_lm_ckpt_to_nemo.py --checkpoint_folder=$PWD --checkpoint_name=$checkpoint_filename --hparams_file=$PWD/hparams.yaml --nemo_file_path=$PWD/biomegatron.nemo --model_type=bert --tensor_model_parallel_size=1

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-01-13 00:21:01 experimental:27] Module <function get_argmin_mat at 0x7f15f2590700> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-13 00:21:01 experimental:27] Module <function getMultiScaleCosAffinityMatrix at 0x7f15f2590790> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-13 00:21:01 experimental:27] Module <function parse_scale_configs at 0x7f15f259d1f0> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022

# Model Training
## Setting up Data within the config

Among other things, the config file contains dictionaries called dataset, train_ds and validation_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.



We assume that both training and evaluation files are located in the same directory, and use the default names mentioned during the data download step. 
So, to start model training, we simply need to specify `model.dataset.data_dir`, like we are going to do below.



Also notice that some config lines, including `model.dataset.data_dir`, have `???` in place of paths, this means that values for these fields are required to be specified by the user.

Let's now add the data directory path to the config.

## Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a Trainer object

In [20]:
WORK_DIR = "WORK_DIR"
os.makedirs(WORK_DIR, exist_ok=True)
MODEL_CONFIG = "token_classification_config.yaml"

In [21]:
# download the model's configuration file 
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

config file is already exists


In [22]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
# Note: these are small batch-sizes - increase as appropriate to available GPU capacity
config.model.train_ds.batch_size=8
config.model.validation_ds.batch_size=8

WORK_DIR/configs/token_classification_config.yaml


In [23]:
# in this tutorial train and dev datasets are located in the same folder, so it is enought to add the path of the data directory to the config
config.model.dataset.data_dir = os.path.join(DATA_DIR, 'NER')

# if you want to decrease the size of your datasets, uncomment the lines below:
# NUM_SAMPLES = 1000
# config.model.train_ds.num_samples = NUM_SAMPLES
# config.model.validation_ds.num_samples = NUM_SAMPLES

In [24]:
print(OmegaConf.to_yaml(config))

pretrained_model: null
trainer:
  gpus: 1
  num_nodes: 1
  max_epochs: 5
  max_steps: null
  accumulate_grad_batches: 1
  gradient_clip_val: 0.0
  precision: 16
  accelerator: ddp
  checkpoint_callback: false
  logger: false
  log_every_n_steps: 1
  val_check_interval: 1.0
  resume_from_checkpoint: null
exp_manager:
  exp_dir: null
  name: token_classification_model
  create_tensorboard_logger: true
  create_checkpoint_callback: true
model:
  label_ids: null
  class_labels:
    class_labels_file: label_ids.csv
  dataset:
    data_dir: DATA_DIR/NER
    class_balancing: null
    max_seq_length: 128
    pad_label: O
    ignore_extra_tokens: false
    ignore_start_end: false
    use_cache: true
    num_workers: 2
    pin_memory: false
    drop_last: false
  train_ds:
    text_file: text_train.txt
    labels_file: labels_train.txt
    shuffle: true
    num_samples: -1
    batch_size: 8
  validation_ds:
    text_file: text_dev.txt
    labels_file: labels_dev.txt
    shuffle: false
    num_sa

In [26]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

Trainer config - 

gpus: 1
num_nodes: 1
max_epochs: 5
max_steps: null
accumulate_grad_batches: 1
gradient_clip_val: 0.0
precision: 16
accelerator: ddp
checkpoint_callback: false
logger: false
log_every_n_steps: 1
val_check_interval: 1.0
resume_from_checkpoint: null



In [27]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

# for PyTorch Native AMP set precision=16
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# remove distributed training flags
config.trainer.accelerator = None

trainer = pl.Trainer(**config.trainer)

Using 16bit native Automatic Mixed Precision (AMP)
[NeMo W 2022-01-13 03:44:12 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py:48: LightningDeprecationWarning: Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7. Use `max_steps = -1` instead.
      rank_zero_deprecation(
    
[NeMo W 2022-01-13 03:44:12 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
      rank_zero_deprecation(
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


## Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [28]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

[NeMo I 2022-01-13 03:44:15 exp_manager:283] Experiments will be logged at /NeMo/tutorials/nlp/nemo_experiments/token_classification_model/2022-01-13_03-44-15
[NeMo I 2022-01-13 03:44:15 exp_manager:648] TensorboardLogger has been set up


[NeMo W 2022-01-13 03:44:15 exp_manager:879] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.
[NeMo W 2022-01-13 03:44:15 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:243: LightningDeprecationWarning: `ModelCheckpoint(every_n_val_epochs)` is deprecated in v1.4 and will be removed in v1.6. Please use `every_n_epochs` instead.
      rank_zero_deprecation(
    


'/NeMo/tutorials/nlp/nemo_experiments/token_classification_model/2022-01-13_03-44-15'

In [29]:
# add the specified above model parameters to the config
# config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.language_model.nemo_file = 'biomegatron.nemo'
config.model.language_model.pretrained_model_name = 'megatron-bert-cased'
config.model.tokenizer.vocab_file='vocab.txt'
config.model.tokenizer.tokenizer_model = 'BertWordPieceCase'
 



Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [30]:
model_ner = nemo_nlp.models.TokenClassificationModel(cfg=config.model, trainer=trainer)

[NeMo I 2022-01-13 03:44:21 tokenizer_utils:124] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-cased, vocab_file: /NeMo/tutorials/nlp/vocab.txt, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


[NeMo I 2022-01-13 03:44:24 token_classification_utils:54] Processing DATA_DIR/NER/labels_train.txt
[NeMo I 2022-01-13 03:44:24 token_classification_utils:90] Labels mapping {'O': 0, 'B-Disease': 1, 'I-Disease': 2} saved to : DATA_DIR/NER/label_ids.csv
[NeMo I 2022-01-13 03:44:24 token_classification_utils:99] Three most popular labels in DATA_DIR/NER/labels_train.txt:
[NeMo I 2022-01-13 03:44:24 data_preprocessing:194] label: 0, 124819 out of 136086 (91.72%).
[NeMo I 2022-01-13 03:44:24 data_preprocessing:194] label: 2, 6122 out of 136086 (4.50%).
[NeMo I 2022-01-13 03:44:24 data_preprocessing:194] label: 1, 5145 out of 136086 (3.78%).
[NeMo I 2022-01-13 03:44:24 token_classification_utils:101] Total labels: 136086. Label frequencies - {0: 124819, 2: 6122, 1: 5145}
[NeMo I 2022-01-13 03:44:24 token_classification_utils:107] Class weights restored from DATA_DIR/NER/labels_train_weights.p
[NeMo I 2022-01-13 03:44:24 token_classification_dataset:277] features restored from DATA_DIR/NER/c

[NeMo W 2022-01-13 03:44:24 modelPT:202] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2022-01-13 03:44:24 lm_utils:78] megatron-bert-cased is not in get_pretrained_lm_models_list(include_external=False), will be using AutoModel from HuggingFace.


[NeMo I 2022-01-13 03:44:35 tokenizer_utils:188] Getting Megatron tokenizer for pretrained model name: megatron-bert-345m-cased and custom vocab file: /NeMo/tutorials/nlp/vocab.txt
[NeMo I 2022-01-13 03:44:35 tokenizer_utils:124] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-cased, vocab_file: /NeMo/tutorials/nlp/vocab.txt, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


[NeMo I 2022-01-13 03:44:38 megatron_bert_model:375] Padded vocab_size: 29056, original vocab_size: 28996, dummy tokens: 60.
[NeMo I 2022-01-13 03:44:40 tokenizer_utils:188] Getting Megatron tokenizer for pretrained model name: megatron-bert-345m-cased and custom vocab file: /NeMo/tutorials/nlp/vocab.txt
[NeMo I 2022-01-13 03:44:40 tokenizer_utils:124] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-cased, vocab_file: /NeMo/tutorials/nlp/vocab.txt, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


[NeMo I 2022-01-13 03:44:43 megatron_bert_model:375] Padded vocab_size: 29056, original vocab_size: 28996, dummy tokens: 60.
[NeMo I 2022-01-13 03:44:44 save_restore_connector:149] Model MegatronBertModel was successfully restored from /NeMo/tutorials/nlp/biomegatron.nemo.


## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.
If you're not using Colab, refer to [https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks) if you're facing issues with running the cell below.

In [31]:
try:
    from google import colab
    COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
    COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
    %load_ext tensorboard
    %tensorboard --logdir {exp_dir}
else:
    print("To use tensorboard, please use this notebook in a Google Colab environment.")

To use tensorboard, please use this notebook in a Google Colab environment.


In [32]:
# start model training
trainer.fit(model_ner)

[NeMo W 2022-01-13 03:45:06 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:287: LightningDeprecationWarning: Base `Callback.on_train_batch_start` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
      rank_zero_deprecation(
    
[NeMo W 2022-01-13 03:45:06 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:287: LightningDeprecationWarning: Base `Callback.on_train_batch_end` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
      rank_zero_deprecation(
    
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4]
[NeMo W 2022-01-13 03:45:06 modelPT:475] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7fe31ee61940>. We recommend to use 'ddp' instead.


[NeMo I 2022-01-13 03:45:06 modelPT:566] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        eps: 1e-08
        lr: 5e-05
        weight_decay: 0.0
    )
[NeMo I 2022-01-13 03:45:06 lr_scheduler:833] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x7fe31d24b0d0>" 
    will be used during training (effective maximum steps = 3395) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    last_epoch: -1
    max_steps: 3395
    )



  | Name                  | Type                 | Params
---------------------------------------------------------------
0 | bert_model            | MegatronBertModel    | 332 M 
1 | classifier            | TokenClassifier      | 1.1 M 
2 | loss                  | CrossEntropyLoss     | 0     
3 | classification_report | ClassificationReport | 0     
---------------------------------------------------------------
333 M     Trainable params
0         Non-trainable params
333 M     Total params
667.288   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2022-01-13 03:45:06 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


...

[NeMo I 2022-01-13 03:49:58 token_classification_model:188] 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         98.13      98.69      98.41      22092
    B-Disease (label_id: 1)                                 68.98      63.28      66.00        787
    I-Disease (label_id: 2)                                 75.29      71.01      73.09       1090
    -------------------
    micro avg                                               96.27      96.27      96.27      23969
    macro avg                                               80.80      77.66      79.17      23969
    weighted avg                                            96.13      96.27      96.19      23969
    


Epoch 3, global step 2715: val_loss reached 0.13950 (best 0.13950), saving model to "/NeMo/tutorials/nlp/nemo_experiments/token_classification_model/2022-01-13_03-44-15/checkpoints/token_classification_model--val_loss=0.1395-epoch=3.ckpt" as top 3


Validating: 0it [00:00, ?it/s]

[NeMo I 2022-01-13 03:51:11 token_classification_model:188] 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         98.34      98.65      98.50      22092
    B-Disease (label_id: 1)                                 66.55      71.79      69.07        787
    I-Disease (label_id: 2)                                 77.87      68.44      72.85       1090
    -------------------
    micro avg                                               96.40      96.40      96.40      23969
    macro avg                                               80.92      79.63      80.14      23969
    weighted avg                                            96.36      96.40      96.36      23969
    


Epoch 4, global step 3394: val_loss reached 0.15355 (best 0.13950), saving model to "/NeMo/tutorials/nlp/nemo_experiments/token_classification_model/2022-01-13_03-44-15/checkpoints/token_classification_model--val_loss=0.1536-epoch=4.ckpt" as top 3
Saving latest checkpoint...


# Inference

To see how the model performs, we can run generate prediction similar to the way we did it earlier

In [33]:
# let's first create a subset of our dev data
! head -n 100 $NER_DATA_DIR/text_dev.txt > $NER_DATA_DIR/sample_text_dev.txt
! head -n 100 $NER_DATA_DIR/labels_dev.txt > $NER_DATA_DIR/sample_labels_dev.txt

Now, let's generate predictions for the provided text file.
If labels file is also specified, the model will evaluate the predictions and plot confusion matrix. 

In [34]:
model_ner.evaluate_from_file(
    text_file=os.path.join(NER_DATA_DIR, 'sample_text_dev.txt'),
    labels_file=os.path.join(NER_DATA_DIR, 'sample_labels_dev.txt'),
    output_dir=exp_dir,
    add_confusion_matrix=False,
    normalize_confusion_matrix=True,
    batch_size=1
)
# Please check matplotlib version if encountering any error plotting confusion matrix:
# https://stackoverflow.com/questions/63212347/importerror-cannot-import-name-png-from-matplotlib

[NeMo I 2022-01-13 03:54:25 token_classification_dataset:121] Setting Max Seq length to: 81
[NeMo I 2022-01-13 03:54:25 data_preprocessing:358] Some stats of the lengths of the sequences:
[NeMo I 2022-01-13 03:54:25 data_preprocessing:360] Min: 4 |                  Max: 81 |                  Mean: 36.03 |                  Median: 33.0
[NeMo I 2022-01-13 03:54:25 data_preprocessing:366] 75 percentile: 47.00
[NeMo I 2022-01-13 03:54:25 data_preprocessing:367] 99 percentile: 68.13


[NeMo W 2022-01-13 03:54:25 token_classification_dataset:150] 0 are longer than 81


[NeMo I 2022-01-13 03:54:25 token_classification_dataset:153] *** Example ***
[NeMo I 2022-01-13 03:54:25 token_classification_dataset:154] i: 0
[NeMo I 2022-01-13 03:54:25 token_classification_dataset:155] subtokens: [CLS] BR ##CA ##1 is secret ##ed and exhibits properties of a g ##rani ##n . [SEP]
[NeMo I 2022-01-13 03:54:25 token_classification_dataset:156] loss_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[NeMo I 2022-01-13 03:54:25 token_classification_dataset:157] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[NeMo I 2022-01-13 03:54:25 token_classification_dataset:158] subtokens_mask: 0 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

RuntimeError: expected scalar type Half but found Float

## Training Script

If you have NeMo installed locally, you can also train the model with `nlp/token_classification/token_classification_train.py.`

To run training script, use:

`python token_classification_train.py model.dataset.data_dir=PATH_TO_DATA_DIR PRETRAINED_BERT_MODEL=biomegatron-bert-345m-cased`

The training could take several minutes and the result should look something like
```
[NeMo I 2020-05-22 17:13:48 token_classification_callback:82] Accuracy: 0.9882348032875798
[NeMo I 2020-05-22 17:13:48 token_classification_callback:86] F1 weighted: 98.82
[NeMo I 2020-05-22 17:13:48 token_classification_callback:86] F1 macro: 93.74
[NeMo I 2020-05-22 17:13:48 token_classification_callback:86] F1 micro: 98.82
[NeMo I 2020-05-22 17:13:49 token_classification_callback:89] precision    recall  f1-score   support
    
    O (label id: 0)     0.9938    0.9957    0.9947     22092
    B (label id: 1)     0.8843    0.9034    0.8938       787
    I (label id: 2)     0.9505    0.8982    0.9236      1090
    
           accuracy                         0.9882     23969
          macro avg     0.9429    0.9324    0.9374     23969
       weighted avg     0.9882    0.9882    0.9882     23969
```